In [1]:
# ------------------------------mount drive-------------------------------------
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/MyDrive/Colab\ Notebooks/nonlinear-sys-id/quadrotor\ neurips/

Mounted at /content/drive/
/content/drive/MyDrive/Colab Notebooks/nonlinear-sys-id/quadrotor neurips


# Getting Trajectory Data of Quadrotor

In [2]:
import numpy as np
from scipy.stats import truncnorm
import random
import matplotlib.pyplot as plt
import math


g = 9.81  # (m/s^2)  gravity constant
dt = 0.01  # time_step for discrete-time system


def system_parameters():
    # -------------------------- Quadcoptor --------------------------
    I_xx = 4.856e-3  # (kg/m^2) moment of inertia
    I_yy = 4.856e-3  # (kg/m^2) moment of inertia
    I_zz = 8.801e-3  # (kg/m^2) moment of inertia
    m = 0.468  # (kg)    weight
    Ax = 0.25  # (kg/s)  drag force coefficients
    Ay = 0.25  # (kg/s)  drag force coefficients
    Az = 0.25  # (kg/s)  drag force coefficients
    # --------------------------- Rotor -------------------------------
    l = 0.225  # (m) distance between the rotor and the center of mass
    k = 2.980e-6  # lift constant of the rotor
    b = 1.140e-7  # drag constant of the rotor
    I_r = 3.357e-5  # (kg/m^2) moment of inertia
    return I_xx, I_yy, I_zz, Ax, Ay, Az, m, l, k, b, I_r


def euler_to_quaternion(roll, pitch, yaw):

    cr = np.cos(roll * 0.5)
    sr = np.sin(roll * 0.5)
    cp = np.cos(pitch * 0.5)
    sp = np.sin(pitch * 0.5)
    cy = np.cos(yaw * 0.5)
    sy = np.sin(yaw * 0.5)

    q0 = cr * cp * cy + sr * sp * sy
    q1 = sr * cp * cy - cr * sp * sy
    q2 = cr * sp * cy + sr * cp * sy
    q3 = cr * cp * sy - sr * sp * cy

    return q0, q1, q2, q3


def generate_u(input_, time_hor, s_, mean, std, u_max, lb, ub):  # noise in control input
    if input_ == "trunc_guass":
        np.random.seed(s_)
        rv = truncnorm(-u_max, u_max, loc=mean, scale=std)
        r1 = rv.rvs(size=time_hor)
        rv = truncnorm(-u_max, u_max, loc=mean, scale=std)
        r2 = rv.rvs(size=time_hor)
        rv = truncnorm(-u_max, u_max, loc=mean, scale=std)
        r3 = rv.rvs(size=time_hor)
        rv = truncnorm(-u_max, u_max, loc=mean, scale=std)
        r4 = rv.rvs(size=time_hor)
        return r1, r2, r3, r4
    elif input_ == "uniform":
        np.random.seed(s_)
        r1 = np.random.uniform(low=lb, high=ub, size=time_hor)
        r2 = np.random.uniform(low=lb, high=ub, size=time_hor)
        r3 = np.random.uniform(low=lb, high=ub, size=time_hor)
        r4 = np.random.uniform(low=lb, high=ub, size=time_hor)
        return r1, r2, r3, r4


def generate_w(distr, time_hor, s_, mean, std, w_max, lb, ub):  # disturbance
    if distr == "trunc_guass":
        np.random.seed(s_)
        rv = truncnorm(-w_max, w_max, loc=mean, scale=std)
        r1 = rv.rvs(size=time_hor)
        rv = truncnorm(-w_max, w_max, loc=mean, scale=std)
        r2 = rv.rvs(size=time_hor)
        rv = truncnorm(-w_max, w_max, loc=mean, scale=std)
        r3 = rv.rvs(size=time_hor)
        rv = truncnorm(-w_max, w_max, loc=mean, scale=std)
        r4 = rv.rvs(size=time_hor)
        rv = truncnorm(-w_max, w_max, loc=mean, scale=std)
        r5 = rv.rvs(size=time_hor)
        rv = truncnorm(-w_max, w_max, loc=mean, scale=std)
        r6 = rv.rvs(size=time_hor)
        return r1, r2, r3, r4, r5, r6
    elif distr == "uniform":
        np.random.seed(s_)
        r1 = np.random.uniform(low=lb, high=ub, size=time_hor)
        r2 = np.random.uniform(low=lb, high=ub, size=time_hor)
        r3 = np.random.uniform(low=lb, high=ub, size=time_hor)
        r4 = np.random.uniform(low=lb, high=ub, size=time_hor)
        r5 = np.random.uniform(low=lb, high=ub, size=time_hor)
        r6 = np.random.uniform(low=lb, high=ub, size=time_hor)
        return r1, r2, r3, r4, r5, r6


class QuadrotorDynamics:
    def __init__(self, distr, input):
        self.distr = distr
        self.input = input
        self.I_xx, self.I_yy, self.I_zz, self.Ax, self.Ay, self.Az, self.m, self.l, self.k, self.b, self.I_r = system_parameters()
        self.px_list = []
        self.py_list = []
        self.pz_list = []
        self.vx_list = []
        self.vy_list = []
        self.vz_list = []
        self.q0_list = []
        self.q1_list = []
        self.q2_list = []
        self.q3_list = []
        self.wx_list = []
        self.wy_list = []
        self.wz_list = []
        self.phi_s_u_list = []
        self.b_s_list = []
        self.u1_lis = []
        self.u2_lis = []
        self.u3_lis = []
        self.u4_lis = []
        self.state_list = []

    def plot_trajectory(self):
        t_list = np.array(range(len(self.px_list))) * dt
        fig = plt.figure()
        plt.plot(t_list, self.px_list, label='$p_{x}$')
        plt.plot(t_list, self.py_list, label='$p_{y}$')
        plt.plot(t_list, self.pz_list, label='$p_{z}$')
        plt.title("Quadrotor's Position")
        plt.xlabel('time (s)')
        plt.ylabel("Quadrotor's Position")
        plt.legend()

        fig = plt.figure()
        plt.plot(t_list, self.vx_list, label='$v_{x}$')
        plt.plot(t_list, self.vy_list, label='$v_{y}$')
        plt.plot(t_list, self.vz_list, label='$v_{z}$')
        plt.title("Quadrotor's Translation Velocity")
        plt.xlabel('time (s)')
        plt.ylabel("Quadrotor's Translation Velocity")
        plt.legend()

        fig = plt.figure()
        plt.plot(t_list, self.q0_list, label='$q_{0}$')
        plt.plot(t_list, self.q1_list, label='$q_{1}$')
        plt.plot(t_list, self.q2_list, label='$q_{2}$')
        plt.plot(t_list, self.q3_list, label='$q_{3}$')
        plt.title("Quaternions")
        plt.xlabel('time (s)')
        plt.ylabel("Quaternion")
        plt.legend()

        fig = plt.figure()
        plt.plot(t_list, self.wx_list, label='$\omega_{x}$')
        plt.plot(t_list, self.wy_list, label='$\omega_{y}$')
        plt.plot(t_list, self.wz_list, label='$\omega_{z}$')
        plt.title("Quadrotor's Angular Velocity")
        plt.xlabel('time (s)')
        plt.ylabel("Quadrotor's Angular Velocity")
        plt.legend()

        fig = plt.figure()
        plt.plot(t_list[1:], self.u1_lis, label='$u_{1}$')
        plt.plot(t_list[1:], self.u2_lis, label='$u_{2}$')
        plt.plot(t_list[1:], self.u3_lis, label='$u_{3}$')
        plt.plot(t_list[1:], self.u4_lis, label='$u_{4}$')
        plt.title("control")
        plt.xlabel('time (s)')
        plt.ylabel("control")
        plt.legend()

        plt.show()

    def update_feature_list(self, phi_s_u, s_, s, ex):
        self.phi_s_u_list.append(phi_s_u)
        self.b_s_list.append(s - s_ - ex)


    def get_trajectory_3(self, x0, time_hor, s_u, s_w, param_u, mult_u, param_w):

        g_ = np.array([0, 0, g])  # gravity vector in inertial frame
        J = np.diag(np.array([self.I_xx, self.I_yy, self.I_zz]))  # inertia matrix
        drag = np.diag(np.array([self.Ax, self.Ay, self.Az]))  # drag force coefficients

        # ----------------------------------------- initial states -----------------------------------------------------
        x = np.array(x0)
        p = x[:3]  # position in inertial frame
        v = x[3:6]  # velocity in inertial frame
        q = x[6:10]  # quaternions
        omega = x[10:]  # angular velocity in body frame

        #  ------------------------------------- Storing the states - ---------------------------------------------
        self.px_list = [p[0]]
        self.py_list = [p[1]]
        self.pz_list = [p[2]]

        self.vx_list = [v[0]]
        self.vy_list = [v[1]]
        self.vz_list = [v[2]]

        self.q0_list = [q[0]]
        self.q1_list = [q[1]]
        self.q2_list = [q[2]]
        self.q3_list = [q[3]]

        self.wx_list = [omega[0]]
        self.wy_list = [omega[1]]
        self.wz_list = [omega[2]]

        self.state_list.append(np.array([p[0], p[1], p[2], v[0], v[1], v[2], q[0], q[1], q[2], q[3], omega[0], omega[1], omega[2]]))

        if self.input == "trunc_guass":
          u_max_ = param_u[2]
        else:
          u_max_ = 1.0

        if self.distr == "trunc_guass":
          w_max_ = param_w[2]
        else:
          w_max_ = 1.0

        # -----------------  random noise and disturbance generation ---------------------------------------------------
        U1_list, U2_list, U3_list, U4_list = generate_u(self.input, time_hor, s_u, mean=param_u[0], std=param_u[1], u_max=u_max_, lb=param_u[0], ub=param_u[1])
        W1_list, W2_list, W3_list, W4_list, W5_list, W6_list = generate_w(self.distr, time_hor, s_w, mean=param_w[0], std=param_w[1], w_max=w_max_, lb=param_w[0], ub=param_w[1])

        # ---------------------------------- controller gains  ------------------------------------------------------
        kp_z = 0.75
        kd_z = 1.25
        kp_phi = 0.03
        kp_theta = 0.03
        kp_psi = 0.03
        kd_phi = 0.00875
        kd_theta = 0.00875
        kd_psi = 0.00875

        # ---------------------------------   desired states   ---------------------------------------------------------
        pz_d = 5.
        vz_d = 0.
        q0_d, q1_d, q2_d, q3_d = euler_to_quaternion(0, 0, 0)

        for t in range(time_hor):

            s_ = np.array([v[0], v[1], v[2], omega[0], omega[1], omega[2]])

            q0 = q[0]
            q1 = q[1]
            q2 = q[2]
            q3 = q[3]

            omega1 = omega[0]
            omega2 = omega[1]
            omega3 = omega[2]

            # ------------------  noise in control input  (for exploration)  ----------------------------------------
            u1 = mult_u[0] * U1_list[t]
            u2 = mult_u[1] * U2_list[t]
            u3 = mult_u[2] * U3_list[t]
            u4 = mult_u[3] * U4_list[t]

            # ----------------   noise in control input  (for exploration)  -----------------------------------------
            w1 = W1_list[t]
            w2 = W2_list[t]
            w3 = W3_list[t]
            w4 = W4_list[t]
            w5 = W5_list[t]
            w6 = W6_list[t]

            # ----------------------------------------  control + noise  ------------------------------------------
            pi_z = kp_z * (pz_d - p[2]) + kd_z * (vz_d - v[2])
            f_c = np.array([0, 0, (5 + pi_z + u1)])

            qe1 = - q0_d * q1 - q3_d * q2 + q2_d * q3 + q1_d * q0
            qe2 = q3_d * q1 - q0_d * q2 - q1_d * q3 + q2_d * q0
            qe3 = - q2_d * q1 + q1_d * q2 - q0_d * q3 + q3_d * q0
            qe4 = q1_d * q1 + q2_d * q2 + q3_d * q3 + q0_d * q0

            pi_phi = -kd_phi * omega1 + kp_phi * qe1 * qe4
            pi_theta = -kd_theta * omega2 + 2 * kp_theta * qe2 * qe4
            pi_psi = -kd_psi * omega3 + 2 * kp_psi * qe3 * qe4
            tau_c = np.array([pi_phi + u2, pi_theta + u3, pi_psi + u4])

            # ------------------------------------------  Dynamic model ----------------------------------------------
            Q = np.array([[q0 * q0 + q1 * q1 - q2 * q2 - q3 * q3, 2 * (q1 * q2 - q0 * q3),
                           2 * (q0 * q2 + q1 * q3)],
                          [2 * (q1 * q2 + q0 * q3), q0 * q0 - q1 * q1 + q2 * q2 - q3 * q3,
                           2 * (q2 * q3 - q0 * q1)],
                          [2 * (q1 * q3 - q0 * q2), 2 * (q0 * q1 + q2 * q3),
                           q0 * q0 - q1 * q1 - q2 * q2 + q3 * q3]])

            Omega = np.array([[0, -omega1, -omega2, -omega3],
                              [omega1, 0, omega3, -omega2],
                              [omega2, -omega3, 0, omega1],
                              [omega3, omega2, -omega1, 0]])

            theta_star = np.array(
                [[1 / self.m, 0., 0., -self.Ax / self.m, 0., 0., 0., 0., 0., 0., 0., 0.],
                 [0., 1 / self.m, 0., 0., -self.Ay / self.m, 0., 0., 0., 0., 0., 0., 0.],
                 [0., 0., 1 / self.m, 0., 0., -self.Az / self.m, 0., 0., 0., 0., 0., 0.],
                 [0., 0., 0., 0., 0., 0., (self.I_yy - self.I_zz) / self.I_xx, 0., 0., 1 / self.I_xx, 0., 0.],
                 [0., 0., 0., 0., 0., 0., 0., (self.I_zz - self.I_xx) / self.I_yy, 0., 0., 1 / self.I_yy, 0.],
                 [0., 0., 0., 0., 0., 0., 0., 0., (self.I_yy - self.I_xx) / self.I_zz, 0., 0., 1 / self.I_zz]])
            Qfc = Q @ f_c
            phi_s_u = np.array([Qfc[0], Qfc[1], Qfc[2],
                                v[0], v[1], v[2],
                                omega2 * omega3, omega1 * omega3, omega1 * omega2,
                                tau_c[0], tau_c[1], tau_c[2]])

            p_dot = v
            q_dot = Omega @ q / 2
            s_dot = - np.array([0., 0., g, 0., 0., 0.]) + theta_star @ phi_s_u + np.array([w1, w2, w3, w4, w5, w6])

            # -------------------------------------- Updating the states --------------------------------------------
            p = p + dt * p_dot
            q = (q + dt * q_dot) / (q0 * q0 + q1 * q1 + q2 * q2 + q3 * q3)
            s = s_ + dt * s_dot
            v = s[:3]
            omega = s[3:]

            self.update_feature_list(dt * phi_s_u, s_, s, - dt * np.array([0., 0., g, 0., 0., 0.]))

            # ------------------------------------- Storing the states ----------------------------------------------
            self.px_list.append(p[0])
            self.py_list.append(p[1])
            self.pz_list.append(p[2])

            self.vx_list.append(v[0])
            self.vy_list.append(v[1])
            self.vz_list.append(v[2])

            self.q0_list.append(q[0])
            self.q1_list.append(q[1])
            self.q2_list.append(q[2])
            self.q3_list.append(q[3])

            self.wx_list.append(omega[0])
            self.wy_list.append(omega[1])
            self.wz_list.append(omega[2])

            self.u1_lis.append(f_c[2])
            self.u2_lis.append(tau_c[0])
            self.u3_lis.append(tau_c[1])
            self.u4_lis.append(tau_c[2])

            self.state_list.append(np.array([p[0], p[1], p[2], v[0], v[1], v[2], q[0], q[1], q[2], q[3], omega[0], omega[1], omega[2]]))



In [3]:
# --------------------------import packages-------------------------------------
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import to_rgba
from scipy.stats import norm
np.set_printoptions(threshold=np.inf)

# --------------------------ground_truth parameters-----------------------------
I_xx, I_yy, I_zz, Ax, Ay, Az, m, _, _, _, _ = system_parameters()
ground_truth = [1 / m, -Ax / m, -Ay / m, -Az / m,
                (I_yy - I_zz) / I_xx, 1 / I_xx,
                (I_zz - I_xx) / I_yy, 1 / I_yy,
                (I_xx - I_yy) / I_zz, 1 / I_zz]

print("-----------------------------------------------------------------------")
print("ground truth = ", ground_truth)
print("-----------------------------------------------------------------------")

n_epoch = 1              # number trajectories
max_time_hor = 100      # maximum trajectory length

# ------------------------------disturbacne-------------------------------------
disturbance: str = "trunc_guass"
parameter_dist = [0.0, 0.1, 10]  # mean and std

# disturbance: str = "uniform"
# parameter_dist = [-1, 1]  # lb and ub

seeds_w = range(300, 500) # fixing seeds
w_max = 0.01             # maximum disturbance (required to run set membership)

# ---------------------------------noise----------------------------------------
c_input: str = "trunc_guass"
parameter_input = [0.0, 0.1, 10]  # mean and std

# c_input: str = "uniform"
# parameter_input = [-1, 1]  # lb and ub

mult_u = [1, 0.2, 0.2, 0.2]

seeds_u = range(100, 200)   # fixing seeds

print('------------------Getting Trajectory Data of Quadrotor-----------------')

theta_hat_list = []
Delta_S_list = []
Phi_S_U_list = []
State_list = []

for e in range(n_epoch):

  # q0, q1, q2, q3 = euler_to_quaternion(10 * np.pi/180, 10 * np.pi/180, 10 * np.pi/180)
  q0 = 1
  q1 = 0
  q2 = 0
  q3 = 0
  x0 = [0., 0., 1., 0., 0., 0., q0, q1, q2, q3, 0., 0., 0.]
  qudt = QuadrotorDynamics(c_input, disturbance)
  qudt.get_trajectory_3(x0, max_time_hor, seeds_u[e], seeds_w[e], parameter_input, mult_u, parameter_dist)
  # print("---------------------------------------------------------------------")
  print("e = ", e + 1)
  # qudt.plot_trajectory()

  Delta_S_list.append(qudt.b_s_list)
  Phi_S_U_list.append(qudt.phi_s_u_list)     # nonlinear feature vector
  State_list.append(qudt.state_list)

print('-------------------------------Data Saved------------------------------')

-----------------------------------------------------------------------
ground truth =  [2.1367521367521367, -0.5341880341880342, -0.5341880341880342, -0.5341880341880342, -0.8123970345963756, 205.9308072487644, 0.8123970345963756, 205.9308072487644, 0.0, 113.62345188046812]
-----------------------------------------------------------------------
------------------Getting Trajectory Data of Quadrotor-----------------
e =  1
-------------------------------Data Saved------------------------------


In [4]:
n_ = 1000
n__ = 12
L_ = np.random.uniform(0,1, size = (n_,n__))
row_sums = L_.sum(axis=1)
L = L_ / row_sums[:, np.newaxis]

In [5]:
def feature_new(state_, phi_, w1, w2, w3, w4, w5, w6, f_, tau1, tau2, tau3):

  s_ = np.array([state_[3], state_[4], state_[5], state_[10], state_[11], state_[12]])
  q_ = [state_[6], state_[7], state_[8], state_[9]]

  I_xx, I_yy, I_zz, Ax, Ay, Az, m, _, _, _, _ = system_parameters()

  theta_star = np.array(
        [[1 / m, 0., 0., -Ax / m, 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1 / m, 0., 0., -Ay / m, 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1 / m, 0., 0., -Az / m, 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., (I_yy - I_zz) / I_xx, 0., 0., 1 / I_xx, 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., (I_zz - I_xx) / I_yy, 0., 0., 1 / I_yy, 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., (I_yy - I_xx) / I_zz, 0., 0., 1 / I_zz]])

  dt = 0.01
  g = 9.8
  s = s_ - dt * np.array([0., 0., g, 0., 0., 0.]) + theta_star @ phi_ + np.array([w1, w2, w3, w4, w5, w6])

  q0 = q_[0]
  q1 = q_[1]
  q2 = q_[2]
  q3 = q_[3]
  Q = np.array([[q0 * q0 + q1 * q1 - q2 * q2 - q3 * q3, 2 * (q1 * q2 - q0 * q3),
                  2 * (q0 * q2 + q1 * q3)],
                [2 * (q1 * q2 + q0 * q3), q0 * q0 - q1 * q1 + q2 * q2 - q3 * q3,
                  2 * (q2 * q3 - q0 * q1)],
                [2 * (q1 * q3 - q0 * q2), 2 * (q0 * q1 + q2 * q3),
                  q0 * q0 - q1 * q1 - q2 * q2 + q3 * q3]])
  Qfc = Q @ np.array([0, 0, f_])
  omega = s[3:]
  omega1 = s[3]
  omega2 = s[4]
  omega3 = s[5]
  phi_new = np.array([Qfc[0], Qfc[1], Qfc[2],
                    s[0], s[1], s[2],
                    omega2 * omega3, omega1 * omega3, omega1 * omega2,
                    tau1, tau2, tau3])

  return phi_new

def estimate_max_prob(L, state_list, phi_list, time_hor_, num_traj, num_iter, c, c_input, disturbance, param_u, mult_u, param_w):

  probs = []
  stds = []
  b_phi = []
  n_ = 0
  e = 0

  if c_input == "trunc_guass":
    u_max_ = param_u[2]
  else:
    u_max_ = 1.0

  if disturbance == "trunc_guass":
    w_max_ = param_w[2]
  else:
    w_max_ = 1.0

  seeds_u = range(100, 100+num_iter*len(L)*time_hor_)
  seeds_w = range(3000, 3000+num_iter*len(L)*time_hor_)

  U1_list, U2_list, U3_list, U4_list = generate_u(c_input, num_iter*len(L)*time_hor_, seeds_u, mean=param_u[0], std=param_u[1], u_max=u_max_, lb=param_u[0], ub=param_u[1])
  W1_list, W2_list, W3_list, W4_list, W5_list, W6_list = generate_w(disturbance, num_iter*len(L)*time_hor_, seeds_w, mean=param_w[0], std=param_w[1], w_max=w_max_, lb=param_w[0], ub=param_w[1])

  ind = 0

  b_phi_ = []
  for tt in range(time_hor_):

    print("-------------------------")
    print("T = ", tt+1)

    for k in range(len(L)):

      l = L[k, :]
      n = 0

      for j in range(num_iter):

        y = feature_new(state_list[e][tt], phi_list[e][tt], W1_list[ind], W2_list[ind] , W3_list[ind], W4_list[ind], W5_list[ind], W6_list[ind],
                        mult_u[0]*U1_list[ind], mult_u[1]*U2_list[ind], mult_u[2]*U3_list[ind], mult_u[3]*U4_list[ind])
        z = np.dot(l, y)
        if (abs(z) <= c):
          n = n + 1

        ind = ind + 1
        b_phi_.append(np.dot(y, y))

      if k == 0:
        b_phi.append(np.mean(np.array(b_phi_)))
      probs.append(n / num_iter)
      n_ = n_ + 1
  print("-------------------------")

  return np.max(np.array(probs)), np.max(np.array(b_phi))


c_list = [1e-3, 5e-3, 1e-2, 5e-2, 0.1, 0.5]
prob_list = []
for c in c_list:
  num_iter = 20
  num_traj = 1
  time_hor_ = 50
  p, b_phi = estimate_max_prob(L, State_list, Phi_S_U_list, time_hor_, num_traj, num_iter, c, c_input, disturbance, parameter_input, mult_u, parameter_dist)
  print("c = ", c)
  print("max prob = ", p)
  print("max b_phi = ", b_phi)
  print("----------------------------------------------------------------------------------------------")
  prob_list.append(p)
print(np.array(c_list) * np.array(prob_list))

-------------------------
T =  1
-------------------------
T =  2
-------------------------
T =  3
-------------------------
T =  4
-------------------------
T =  5
-------------------------
T =  6
-------------------------
T =  7
-------------------------
T =  8
-------------------------
T =  9
-------------------------
T =  10
-------------------------
T =  11
-------------------------
T =  12
-------------------------
T =  13
-------------------------
T =  14
-------------------------
T =  15
-------------------------
T =  16
-------------------------
T =  17
-------------------------
T =  18
-------------------------
T =  19
-------------------------
T =  20
-------------------------
T =  21
-------------------------
T =  22
-------------------------
T =  23
-------------------------
T =  24
-------------------------
T =  25
-------------------------
T =  26
-------------------------
T =  27
-------------------------
T =  28
-------------------------
T =  29
-----------------------

In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt

eps = 0.1
T = 30000

b_phi = 2
n_x = 6
n_u = 4
n_phi = 12

sigma_u = 0.5
sigma_w = 0.005
w_max = 0.01
s_phi = 0.05
p_phi = 0.95


a1 = s_phi * p_phi / 4
a2 = (8 * b_phi / (s_phi * p_phi) ) ** 2
a3 = (p_phi ** 2) / 8
a4 = max (1, (4 * b_phi * np.sqrt(n_x) / a1))

m = (1 / a3) * (np.log(T/eps) + n_phi*np.log(a2) + 2.5*np.log(n_phi) + np.log(np.log(a2 * n_phi)) + 7)
print(m)
m_ = int(np.ceil(m))
theo_diam = []
a = ((n_phi) ** 2.5)  * (a2 ** n_phi)  * np.exp(-a3*m)
b = ((n_phi * n_x) ** 2.5)  * (a4 ** (n_phi * n_x))
for t in range(m_, T):
  diam_= 4 / t
  theo_diam.append(diam_)

disturbance = "trunc_"
name1 = 'sme_theo_bound_1_' + 'w_' + disturbance + '_' + str(sigma_w) + '_' + 's_phi' '_' + str(s_phi) + '_' + 'p_phi' +  '_' + str(p_phi) +  '_' + 'eps' +   '_' + str(eps) + '.csv'
name2 = 'sme_theo_bound_2_' + 'w_' + disturbance + '_' + str(sigma_w) + '_' + 's_phi' '_' + str(s_phi) + '_' + 'p_phi' +  '_' + str(p_phi) +  '_' + 'eps' +   '_' + str(eps) + '.csv'
np.savetxt(name1, np.array(range(m_, T)), delimiter = ",")
np.savetxt(name2, 1000 * np.array(theo_diam), delimiter = ",")

1490.4592345260874


In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt

eps = 0.1
T = 30000

n_x = 6
n_u = 4
n_phi = 12

w_max = 0.01
u_max = 0.01
sigma_u = np.sqrt((2*u_max)**2/12)
sigma_w = np.sqrt((2*w_max)**2/12)

s_phi = 0.05
p_phi = 0.8
b_phi = np.sqrt(5)


a1 = s_phi * p_phi / 4
a2 = (8 * b_phi / (s_phi * p_phi) ) ** 2
a3 = (p_phi ** 2) / 8
a4 = max (1, (4 * b_phi * np.sqrt(n_x) / a1))

m = (1 / a3) * (np.log(T/eps) + n_phi*np.log(a2) + 2.5*np.log(n_phi) + np.log(np.log(a2 * n_phi)) + 7)
print(m)
m_ = int(np.ceil(m))
theo_diam = []
a = ((n_phi) ** 2.5)  * (a2 ** n_phi)  * np.exp(-a3*m)
b = ((n_phi * n_x) ** 2.5)  * (a4 ** (n_phi * n_x))
for t in range(m_, T):
  diam_= 4 / t
  theo_diam.append(diam_)

disturbance = "uniform_"
name1 = 'sme_theo_bound_1_' + 'w_' + disturbance + '_' + str(sigma_w) + '_' + 's_phi' '_' + str(s_phi) + '_' + 'p_phi' +  '_' + str(p_phi) +  '_' + 'eps' +   '_' + str(eps) + '.csv'
name2 = 'sme_theo_bound_2_' + 'w_' + disturbance + '_' + str(sigma_w) + '_' + 's_phi' '_' + str(s_phi) + '_' + 'p_phi' +  '_' + str(p_phi) +  '_' + 'eps' +   '_' + str(eps) + '.csv'
np.savetxt(name1, np.array(range(m_, T)), delimiter = ",")
np.savetxt(name2, 1000 * np.array(theo_diam), delimiter = ",")

2187.2988722064138


In [6]:
delta = 0.1
time_hor = 30000

n_x = 6
n_u = 4
n_phi = 12

sigma_u = 0.001
sigma_w = 0.001
w_max = 0.01

s_phi = 0.01
p_phi = 0.95

b_phi = 2

theo_bound = []
time_list = []
for k in range(time_hor):
  cc = b_phi/delta/s_phi/s_phi
  cond = (10/p_phi) * (np.log(1/delta) + 2*n_phi*np.log(10/p_phi) + n_phi*np.log(cc))
  if (k+1) >= cond:
    time_list.append(k+1)
    theo_bound_ = (90*sigma_w/p_phi) * np.sqrt((n_x + np.log(1/delta) + n_phi*np.log(10/p_phi) + n_phi*np.log(cc)) / ((k+1)*s_phi))
    theo_bound.append(theo_bound_)

disturbance = "trunc_guass"
name1 = 'lse_theo_bound_1_' + 'w_' + disturbance + '_' + str(sigma_w) + '_' + 's_phi' '_' + str(s_phi) + '_' + 'p_phi' +  '_' + str(p_phi) +  '_' + 'delta' +   '_' + str(delta) + '.csv'
name2 = 'lse_theo_bound_2_' + 'w_' + disturbance + '_' + str(sigma_w) + '_' + 's_phi' '_' + str(s_phi) + '_' + 'p_phi' +  '_' + str(p_phi) +  '_' + 'delta' +   '_' + str(delta) + '.csv'
np.savetxt(name1, np.array(time_list), delimiter = ",")
np.savetxt(name2, np.array(theo_bound), delimiter = ",")

In [7]:
print(name1)

lse_theo_bound_1_w_trunc_guass_0.001_s_phi_0.01_p_phi_0.95_delta_0.1.csv


In [8]:
delta = 0.1
time_hor = 30000

w_max = 0.01
u_max = 0.01
sigma_u = np.sqrt((2*u_max)**2/12)
sigma_w = np.sqrt((2*w_max)**2/12)

s_phi = 0.05
p_phi = 0.8
b_phi = 5

theo_bound = []
time_list = []
for k in range(time_hor):
  cc = b_phi/delta/s_phi/s_phi
  cond = (10/p_phi) * (np.log(1/delta) + 2*n_phi*np.log(10/p_phi) + n_phi*np.log(cc))
  if (k+1) >= cond:
    time_list.append(k+1)
    theo_bound_ = (90*sigma_w/p_phi) * np.sqrt((n_x + np.log(1/delta) + n_phi*np.log(10/p_phi) + n_phi*np.log(cc)) / ((k+1)*s_phi))
    theo_bound.append(theo_bound_)

disturbance = "uniform"
name1 = 'lse_theo_bound_1_' + 'w_' + disturbance + '_' + str(sigma_w) + '_' + 's_phi' '_' + str(s_phi) + '_' + 'p_phi' +  '_' + str(p_phi) +  '_' + 'delta' +   '_' + str(delta) + '.csv'
name2 = 'lse_theo_bound_2_' + 'w_' + disturbance + '_' + str(sigma_w) + '_' + 's_phi' '_' + str(s_phi) + '_' + 'p_phi' +  '_' + str(p_phi) +  '_' + 'delta' +   '_' + str(delta) + '.csv'
np.savetxt(name1, np.array(time_list), delimiter = ",")
np.savetxt(name2, np.array(theo_bound), delimiter = ",")

In [9]:
print(name1)

lse_theo_bound_1_w_uniform_0.005773502691896258_s_phi_0.05_p_phi_0.8_delta_0.1.csv


In [ ]:
delta = 0.1
time_hor = 30000

n_x = 6
n_u = 4
n_phi = 12

sigma_u = 0.1
sigma_w = 0.001
w_max = 0.01
s_phi = 0.01
p_phi = 0.95
b_phi = 2

theo_bound = []
time_list = []
for k in range(time_hor):
  cc = b_phi/delta/s_phi/s_phi
  cond = (10/p_phi) * (np.log(1/delta) + 2*n_phi*np.log(10/p_phi) + n_phi*np.log(cc))
  if (k+1) >= cond:
    time_list.append(k+1)
    theo_bound_ = (90*sigma_w/p_phi) * np.sqrt((n_x + np.log(1/delta) + n_phi*np.log(10/p_phi) + n_phi*np.log(cc)) / ((k+1)*s_phi))
    theo_bound.append(theo_bound_)

disturbance = "trunc_guass"
name1 = 'lse_theo_bound_1_' + 'w_' + disturbance + '_' + str(sigma_w) + '_' + 's_phi' '_' + str(s_phi) + '_' + 'p_phi' +  '_' + str(p_phi) +  '_' + 'delta' +   '_' + str(delta) + '.csv'
name2 = 'lse_theo_bound_2_' + 'w_' + disturbance + '_' + str(sigma_w) + '_' + 's_phi' '_' + str(s_phi) + '_' + 'p_phi' +  '_' + str(p_phi) +  '_' + 'delta' +   '_' + str(delta) + '.csv'
np.savetxt(name1, np.array(time_list), delimiter = ",")
np.savetxt(name2, np.array(theo_bound), delimiter = ",")